<a href="https://colab.research.google.com/github/Christian-Martens-UNCC/ECGR-4106_Final-Project/blob/main/ECGR4106_FinalProject_ntm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/final_project_datasets.zip', 'r')
zip_ref.extractall('')
zip_ref.close()

In [ ]:
column_types = [tf.float64, tf.float64, tf.float64, tf.float64, tf.float64, tf.int64]
dataset_pro_raw = tf.data.Dataset.list_files([f"./stocks/*.csv"], shuffle=True)

dataset_pro_i = dataset_pro_raw.interleave(
  lambda file: tf.data.experimental.CsvDataset(file, column_types, header=True),
  # how many files should be processed concurently
  cycle_length = 20,
  num_parallel_calls = 10
)

In [ ]:
import pandas as pd

In [ ]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
ntm_train = pd.read_csv("./stocks/A.csv")

ntm_train.head()
M = len(ntm_train)
M

5124

In [ ]:
ntm_data = pd.DataFrame(ntm_train)
ntm_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300
1,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100
2,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800
3,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600
4,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200


In [ ]:
m = len(ntm_data)
m

5124

In [ ]:
ntm_data.shape

(5124, 7)

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(0)
ntm_train, ntm_test = train_test_split(ntm_data, train_size=0.8, test_size=0.2)
ntm_train.shape

(4099, 7)

In [ ]:
ntm_test.shape

(1025, 7)

In [ ]:
num_vars = ['Open', 'High', 'Low', 'Volume',
            'Close', 'Adj Close']
ntm_NewTrain = ntm_train[num_vars]
ntm_NewTest = ntm_test[num_vars]
ntm_NewTrain.head()

,Open,High,Low,Volume,Close,Adj Close
1319,16.959942,17.002861,16.788269,2814500,16.859800,14.500194
3838,41.619999,42.110001,41.599998,1922100,42.060001,39.979748
70,74.749641,79.935623,74.570816,1037800,78.057938,67.133362
2493,19.556509,19.706724,19.363377,2850100,19.599428,17.890099
4400,56.830002,56.830002,55.669998,1839700,55.770000,54.297264


In [ ]:
ntm_NewTrain.shape

(4099, 6)

In [ ]:
ntm_NewTest.shape

(1025, 6)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()
ntm_NewTrain.head()

,Open,High,Low,Volume,Close,Adj Close
1319,16.959942,17.002861,16.788269,2814500,16.859800,14.500194
3838,41.619999,42.110001,41.599998,1922100,42.060001,39.979748
70,74.749641,79.935623,74.570816,1037800,78.057938,67.133362
2493,19.556509,19.706724,19.363377,2850100,19.599428,17.890099
4400,56.830002,56.830002,55.669998,1839700,55.770000,54.297264


In [ ]:
Y = ntm_NewTrain.values[:, 4]
print('Y = ', Y[: 5])

Y =  [16.86  42.06  78.058 19.599 55.77 ]


In [ ]:
ntm_features = ntm_NewTrain.copy()
ntm_labels = ntm_features.pop('Close')

In [ ]:
from keras.layers.core import Dense
from keras.layers import LSTM
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.initializers import RandomNormal

In [ ]:
output_dim = 1
input_dim = 5
batch_size = 100
read_heads = 1
write_heads = 1

In [ ]:
zip_ref = zipfile.ZipFile('drive/MyDrive/myFolder.zip', 'r')
zip_ref.extractall('')
zip_ref.close()

In [ ]:
import model_ntm
from ntm import controller_input_output_shape as controller_shape

In [ ]:
from ntm_keras-master import model_ntm
from ntm import controller_input_output_shape as controller_shape

SyntaxError: ignored

In [ ]:
controller_input_dim, controller_output_dim = controller_shape(input_dim, output_dim, 20, 128, 3, read_heads, write_heads)

In [ ]:
sameInit = RandomNormal(seed=0)
controller = Sequential() 
controller.add(Dense(units=controller_output_dim,
                     kernel_initializer=sameInit,
                     bias_initializer=sameInit,
                     activation='linear',
                     input_dim=controller_input_dim))

In [ ]:
sgd = Adam(lr=5e-4, clipnorm=10)
controller.compile(loss='binary_cross_entropy', optimizer=sgd, metrics=['binary_accuracy'], sample_weight_mode="temporal")

model = model_ntm.gen_model(input_dim=input_dim, output_dim=output_dim, batch_size=batch_size, controller_model=controller, read_heads=read_heads, write_heads=write_heads,
                            activation="sigmoid")

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


AttributeError: ignored

In [ ]:
ntm_train.columns[5]

'Volume'

In [ ]:
dataset = tf.data.experimental.CsvDataset(
    "./stocks/AAMC.csv",
    [tf.string, tf.float64, tf.float64, tf.float64,
     tf.float64, tf.float64, tf.int64],
     select_cols=[1,2,3,4,5,6]
)

InvalidArgumentError: ignored